<a href="https://colab.research.google.com/github/rrsartneoai/Awesome-Multimodal-Large-Language-Models/blob/main/Kopia_notatnika_quicktour.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Transformers installation
! pip install transformers datasets
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer, Trainer, TrainingArguments

from datasets import load_dataset, load_metric

In [ ]:
dataset = load_dataset('text', data_files={'train': 'train.txt', 'test': 'test.txt'})

In [ ]:
# Inicjalizacja tokenizer'a i modelu

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Tokenizacja danych

def tokenize_function(examples):
    # Indent the return statement to be part of the function
    return tokenizer(examples['text'], padding='max_length', truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
# Parametry fine-tuningu

training_args = TrainingArguments(

output_dir='./results',

num_train_epochs=3, # Liczba epok

per_device_train_batch_size=16, # Batch size

per_device_eval_batch_size=16,

learning_rate=2e-5, # Learning rate

evaluation_strategy="epoch",

logging_dir='./logs',

)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
import numpy as np
from transformers import Trainer, TrainingArguments
from datasets import load_metric

# Load the metric
metric = load_metric("accuracy")  # Or any other suitable metric

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"].add_column('labels', [0]*len(tokenized_datasets["train"])),  # Add dummy labels for demonstration
    eval_dataset=tokenized_datasets["test"].add_column('labels', [0]*len(tokenized_datasets["test"])),  # Add dummy labels for demonstration
    compute_metrics=compute_metrics,
)

# Start the training process
trainer.train()

<ipython-input-17-7d30c1256e3e>:6: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")  # Or any other suitable metric


The repository for accuracy contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/accuracy.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.325784,1.000000
2,No log,0.309726,1.000000
3,No log,0.295531,1.000000


TrainOutput(global_step=3, training_loss=0.30373034874598187, metrics={'train_runtime': 64.385, 'train_samples_per_second': 0.093, 'train_steps_per_second': 0.047, 'total_flos': 1578666332160.0, 'train_loss': 0.30373034874598187, 'epoch': 3.0})